In [1]:
# Set annotation to annot_ce10
%run ~/relmapping/annot/notebooks/__init__.ipynb
annot = 'annot_ce10'
def vp(fp): return os.path.join(config[annot]['loc_metrics'], fp)

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/beegfs/scratch_copy/ahringer/jj374/lab/relmapping


In [ ]:
# Load long cap jump & exon data
df_atac = pd.read_csv(config[annot]['fp_sites_tsv'], sep='\t')[yp.NAMES_BED3]
l_atac_peak_pos = df_atac[['start', 'end']].mean(axis=1).map(int)
df_atac.head()

,chrom,start,end
0,chrI,1900,2051
1,chrI,3826,3977
2,chrI,4276,4427
3,chrI,11272,11423
4,chrI,13070,13221


In [ ]:
# scap_count <- <scap tags within region>; scap_mode_raw <- <scap mode position>; scap_mode_count <- <tags at mode>
fp_fwd = pf('scap815_wt_all', 'tg_se.bwa_se.rm_unmapped.rm_chrM.rm_blacklist.rm_non_coding.rm_q10.firstbp_fwd.gt0x2', '.bw', 'scap815')
fp_rev = pf('scap815_wt_all', 'tg_se.bwa_se.rm_unmapped.rm_chrM.rm_blacklist.rm_non_coding.rm_q10.firstbp_rev.gt0x2', '.bw', 'scap815')

flank_len = 125
chroms_ = list(df_atac['chrom'])
starts_ = list(l_atac_peak_pos - flank_len)
ends_ =  list(l_atac_peak_pos + flank_len)

df_scap_fwd = pd.DataFrame()
df_scap_rev = pd.DataFrame()
df_scap_fwd['scap_count'] = [*map(lambda c: int(np.nansum(c)), yp.read_regions(fp_fwd, chroms_, starts_, ends_))]
df_scap_rev['scap_count'] = [*map(lambda c: int(np.nansum(c)), yp.read_regions(fp_rev, chroms_, starts_, ends_))]

def scap_mode_count_(c): return int(np.nan_to_num(np.nanmax(c)))
df_scap_fwd['scap_mode_count'] = [*map(scap_mode_count_, yp.read_regions(fp_fwd, chroms_, starts_, ends_))]
df_scap_rev['scap_mode_count'] = [*map(scap_mode_count_, yp.read_regions(fp_rev, chroms_, starts_, ends_))]

df_scap_fwd['scap_pass'] = df_scap_fwd['scap_mode_count'] >= 2
df_scap_rev['scap_pass'] = df_scap_rev['scap_mode_count'] >= 2

n = float(len(df_atac))
n_scap_fwd = len(df_scap_fwd.query('scap_mode_count >= 2'))
n_scap_rev = len(df_scap_rev.query('scap_mode_count >= 2'))
n_scap_bid = sum((df_scap_fwd['scap_mode_count'] >= 2) & (df_scap_rev['scap_mode_count'] >= 2))
n_scap_either = sum((df_scap_fwd['scap_mode_count'] >= 2) | (df_scap_rev['scap_mode_count'] >= 2))
print(n_scap_fwd, n_scap_rev, n_scap_bid, n_scap_either, 'fwd/rev/both/either in pooled scap')
print(n_scap_fwd / n, n_scap_rev / n, n_scap_bid / n, n_scap_either / n)

In [ ]:
# "Extrapolate" likely regions of txn initiation based on peak accessibility
# On average, short cap modes are spaced by 120bp
# Extrapolate short cap modes by peak accessibility +/- 60 (120bp/2) if no short cap mode was found
# (These "extrapolated" short cap modes can be discarded by filtering for scap_mode_count > 0)
def scap_mode_fwd_(scap_mode_raw, scap_mode_count, atac_mode):
    if scap_mode_count > 0:
        return scap_mode_raw
    else:
        return atac_mode + 60

def scap_mode_rev_(scap_mode_raw, scap_mode_count, atac_mode):
    if scap_mode_count > 0:
        return scap_mode_raw
    else:
        return atac_mode - 60

def scap_mode_raw_(s, c): return int(s + yp.nanargmax_median(c))
df_scap_fwd['scap_mode_raw'] = [*map(scap_mode_raw_, starts_, yp.read_regions(fp_fwd, chroms_, starts_, ends_))]
df_scap_rev['scap_mode_raw'] = [*map(scap_mode_raw_, starts_, yp.read_regions(fp_rev, chroms_, starts_, ends_))]

df_scap_fwd['scap_mode'] = [*map(scap_mode_fwd_, df_scap_fwd['scap_mode_raw'], df_scap_fwd['scap_mode_count'], l_atac_peak_pos)]
df_scap_rev['scap_mode'] = [*map(scap_mode_rev_, df_scap_rev['scap_mode_raw'], df_scap_rev['scap_mode_count'], l_atac_peak_pos)]

m = (df_scap_fwd['scap_mode_count'] >= 2) & (df_scap_rev['scap_mode_count'] >= 2)
print(np.median(df_scap_fwd[m]['scap_mode_raw'] - df_scap_rev[m]['scap_mode_raw']), 'median mode >= 2 tags')
m = (df_scap_fwd['scap_mode_count'] >= 10) & (df_scap_rev['scap_mode_count'] >= 10)
print(np.median(df_scap_fwd[m]['scap_mode_raw'] - df_scap_rev[m]['scap_mode_raw']), 'median mode (>= 10 tags)')

print('%d / %d sites with extrapolated txn initiation' % (
    len(df_scap_fwd.query('scap_mode_count == 0')), 
    len(df_scap_rev.query('scap_mode_count == 0'))
))

In [ ]:
# scap_%strand.bed <- txn initiation statistics
def itemRgb_(flag): return yp.ORANGE if flag else yp.BLUE

l_scap_col = ['scap_count', 'scap_mode_count', 'scap_pass', 'scap_mode_raw', 'scap_mode']

write_gffbed(vp('scap_fwd.bed'),
    chrom = df_atac['chrom'],
    start = list(l_atac_peak_pos - flank_len),
    end = list(l_atac_peak_pos + flank_len),
    name = df_scap_fwd['scap_mode_count'],
    strand = '+',
    attr = df_scap_fwd[l_scap_col],
    itemRgb = map(itemRgb_, df_scap_fwd['scap_pass']),
)

write_gffbed(vp('scap_rev.bed'),
    chrom = df_atac['chrom'],
    start = list(l_atac_peak_pos - flank_len),
    end = list(l_atac_peak_pos + flank_len),
    name = df_scap_rev['scap_mode_count'],
    strand = '-',
    attr = df_scap_rev[l_scap_col],
    itemRgb = map(itemRgb_, df_scap_rev['scap_pass']),
)

df_scap_fwd[l_scap_col].to_csv(vp('scap_fwd.tsv'), header=True, index=False, sep='\t')
df_scap_rev[l_scap_col].to_csv(vp('scap_rev.tsv'), header=True, index=False, sep='\t')